# FSRS Parameter Divergence Exploration

This notebook explores the divergence between 'Nature' (the ground truth stability) and the FSRS model predictions under various scenarios. 

## Setup
We import the core simulation engine and optimized metric calculation logic.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm.auto import tqdm
import concurrent.futures
import multiprocessing

# Add src to path for imports
sys.path.append(os.path.abspath('src'))

from simulate_fsrs import run_simulation, DEFAULT_PARAMETERS, load_anki_history, Card, Scheduler, infer_review_weights
from plot_fsrs_divergence import calculate_metrics, calculate_population_retrievability, init_worker

# Shared configuration
REPEATS = 5
CONCURRENCY = max(1, multiprocessing.cpu_count() // 2)
SEED_HISTORY = 'collection.anki2' if os.path.exists('collection.anki2') else None
DECK_CONFIG = 'Puzzles'
T_EVAL = np.linspace(0, 100, 200)

## Helper Functions
Functions to run batches of simulations and plot the results.

In [ ]:
def run_scenario(tasks):
    results = defaultdict(list)
    
    # Pre-load seeded payload for workers
    seeded_payload = None
    if SEED_HISTORY:
        logs, last_rev = load_anki_history(SEED_HISTORY, deck_config_name=DECK_CONFIG)
        if logs:
            weights = infer_review_weights(logs)
            nat_sch = Scheduler(parameters=DEFAULT_PARAMETERS)
            alg_sch = Scheduler(parameters=DEFAULT_PARAMETERS)
            true_cards = {}
            sys_cards = {}
            for cid, card_logs in logs.items():
                true_cards[cid] = nat_sch.reschedule_card(Card(card_id=cid), card_logs)
                sys_cards[cid] = alg_sch.reschedule_card(Card(card_id=cid), card_logs)
            
            seeded_payload = {
                'logs': logs,
                'last_rev': last_rev,
                'true_cards': true_cards,
                'sys_cards': sys_cards,
                'weights': weights
            }

    with concurrent.futures.ProcessPoolExecutor(
        max_workers=CONCURRENCY,
        initializer=init_worker,
        initargs=(seeded_payload,)
    ) as executor:
        future_to_task = {}
        for task_label, task_params in tasks.items():
            for i in range(REPEATS):
                t = dict(task_params)
                t['seed'] = 42 + i
                t['verbose'] = False
                t['seeded_data'] = None
                future = executor.submit(run_simulation, **t)
                future_to_task[future] = task_label

        for future in tqdm(concurrent.futures.as_completed(future_to_task), total=len(future_to_task)):
            label = future_to_task[future]
            try:
                fitted, gt, metrics = future.result()
                if fitted:
                    stabilities = metrics['stabilities']
                    rmse, kl = calculate_metrics(gt, fitted, stabilities)
                    
                    s_nat = np.array([s[0] for s in stabilities])
                    s_alg = np.array([s[1] for s in stabilities])
                    r_nat = calculate_population_retrievability(T_EVAL, s_nat, gt)
                    r_fit = calculate_population_retrievability(T_EVAL, s_alg, fitted)
                    
                    results[label].append({
                        'rmse': rmse,
                        'kl': kl,
                        'r_nat': r_nat,
                        'r_fit': r_fit
                    })
            except Exception as e:
                print(f'Task {label} failed: {e}')
    return results

def plot_results(scenario_results, title):
    plt.figure(figsize=(12, 7))
    for label, repeats in scenario_results.items():
        avg_fit = np.mean([r['r_fit'] for r in repeats], axis=0)
        avg_nat = np.mean([r['r_nat'] for r in repeats], axis=0)
        avg_rmse = np.mean([r['rmse'] for r in repeats])
        avg_kl = np.mean([r['kl'] for r in repeats])
        
        line, = plt.plot(T_EVAL, avg_fit, label=f'{label} (RMSE: {avg_rmse:.4f}, KL: {avg_kl:.4f})')
        plt.plot(T_EVAL, avg_nat, linestyle='--', color=line.get_color(), alpha=0.5)
        
    plt.title(title)
    plt.xlabel('Days since end of simulation')
    plt.ylabel('Aggregate Expected Retention')
    plt.legend(fontsize='small', ncol=1)
    plt.grid(True, alpha=0.3)
    plt.show()

## Scenario 1: Duration Exploration
Default parameters, 0.9 retention for different time periods (30, 90, 180, 365 days).

In [ ]:
tasks_1 = {
    '30 Days': {'n_days': 30, 'retention': '0.9'},
    '90 Days': {'n_days': 90, 'retention': '0.9'},
    '180 Days': {'n_days': 180, 'retention': '0.9'},
    '365 Days': {'n_days': 365, 'retention': '0.9'}
}
res_1 = run_scenario(tasks_1)
plot_results(res_1, 'Scenario 1: Impact of Simulation Duration (0.9 Retention)')

## Scenario 2: Retention Contrast
Default parameters, 0.9 vs 0.7 retention for the same time period (180 days).

In [ ]:
tasks_2 = {
    'Ret=0.9': {'n_days': 180, 'retention': '0.9'},
    'Ret=0.7': {'n_days': 180, 'retention': '0.7'}
}
res_2 = run_scenario(tasks_2)
plot_results(res_2, 'Scenario 2: Impact of Target Retention (180 Days)')

## Scenario 3: Static 0.9 vs Variable Schedule
Default params, static 0.9 vs a 5:1 weighted schedule favoring 0.9.

In [ ]:
tasks_3 = {
    'Static 0.9': {'n_days': 180, 'retention': '0.9'},
    'Sched 0.9:5,0.7:1': {'n_days': 180, 'retention': '5:0.9,1:0.7'}
}
res_3 = run_scenario(tasks_3)
plot_results(res_3, 'Scenario 3: Static 0.9 vs 0.9:5,0.7:1 Schedule')

## Scenario 4: Static 0.7 vs Variable Schedule
Default params, static 0.7 vs a 5:1 weighted schedule favoring 0.7.

In [ ]:
tasks_4 = {
    'Static 0.7': {'n_days': 180, 'retention': '0.7'},
    'Sched 0.7:5,0.9:1': {'n_days': 180, 'retention': '5:0.7,1:0.9'}
}
res_4 = run_scenario(tasks_4)
plot_results(res_4, 'Scenario 4: Static 0.7 vs 0.7:5,0.9:1 Schedule')

## Scenario 5: High-Retention Burn-In
Default params, 60 days of 0.9 burn-in followed by a shift to 0.7 vs staying at 0.9.

In [ ]:
tasks_5 = {
    '0.9 BI -> 0.7 post': {'n_days': 180, 'burn_in_days': 60, 'retention': '60:0.9,120:0.7'}
    '0.9 BI -> 0.7 post': {'n_days': 180, 'burn_in_days': 60, 'retention': '60:0.9,120:0.7'},
}
res_5 = run_scenario(tasks_5)
plot_results(res_5, 'Scenario 5: Post-Burn-In Retention Shift (0.9 Initial)')

## Scenario 6: Low-Retention Burn-In
Default params, 60 days of 0.7 burn-in followed by a shift to 0.9 vs staying at 0.7.

In [ ]:
tasks_6 = {
    '0.7 BI -> 0.7 post': {'n_days': 180, 'burn_in_days': 60, 'retention': '0.7'}
    '0.7 BI -> 0.7 post': {'n_days': 180, 'burn_in_days': 60, 'retention': '0.7'},
}
res_6 = run_scenario(tasks_6)
plot_results(res_6, 'Scenario 6: Post-Burn-In Retention Shift (0.7 Initial)')